In [ ]:
import pandas as pd             # Data analysis
import numpy as np              # Transform data
from datetime import timedelta
#import talib
#from datetime import timedelta

class GoogleTrendTrading(QCAlgorithm):

    def Initialize(self):
        self.length = 10
        # InteractiveBrokers: Supported Order Type = Market Order, Limit Order, Stop Market, Stop Limit Order, Market On Open, Market on Close
        self.SetBrokerageModel(BrokerageName.InteractiveBrokersBrokerage)       # Documentation: /algorithm-reference/reality-modelling
        self.SetStartDate(2018, 1, 1)                                           # Set Start Date
        self.SetEndDate(2020, 9, 30)                                            # Set End Date
        self.SetCash(100000)   # Set Cash to $100,000
        self.SetWarmUp(self.length)

        self.AddForex("EURUSD", Resolution.Minute)
      
        self.Securities["EURUSD"].SetDataNormalizationMode(DataNormalizationMode.Raw);
        self.__macd = self.MACD("EURUSD", 12, 20, 4, MovingAverageType.Exponential,Resolution.Hour)#MovingAverageType.Exponential
        
        self.__macd_min = self.MACD("EURUSD", 12, 20, 4, MovingAverageType.Exponential,Resolution.Minute)
        self.atr = self.ATR("EURUSD", 5, MovingAverageType.Simple, Resolution.Minute)
        
        #self.adx = self.ADX("EURUSD",5, Resolution.Minute)
        self.adx = self.ADX("EURUSD",20, Resolution.Hour)
       # self.__macd_daily = self.MACD("EURUSD", 12, 26, 9, MovingAverageType.Exponential, Resolution.Daily)
       # self.bool =self.BB("EURUSD",20,2,Resolution.Minute)
        self.bool_m =self.BB("EURUSD",20,2,Resolution.Minute)
        self.bool_hr =self.BB("EURUSD",20,2,Resolution.Hour)
        self.bool_4hr =self.BB("EURUSD",20,2,Resolution.Hour)
        self.window = RollingWindow[QuoteBar](20)    # For other security types, use QuoteBar
        
        self.windowmin = RollingWindow[QuoteBar](20)
        self.window_4hr=RollingWindow[QuoteBar](20)
       # self.lwma_daily = self.LWMA("EURUSD",20,Resolution.Daily)
        #self.lwma = self.LWMA("EURUSD",20,Resolution.Minute)
       # self.lwma_hr = self.LWMA("EURUSD",20,Resolution.Hour)
       # self.lwma_4hr = self.LWMA("EURUSD",20,Resolution.Hour)
        # Creates an indicator and adds to a rolling window when it is updated
        self.__macd_min.Updated +=self.MacdUpdatedMin
        self._macdWin2 = RollingWindow[IndicatorDataPoint](20)
        #self.sma = self.SMA("SPY", 5)
        self.__macd.Updated +=self.MacdUpdated
        self._macdWin = RollingWindow[IndicatorDataPoint](20)
        #self.pattern = self.CandlestickPatterns.Doji("EURUSD",Resolution.Hour)#CandlestickPatterns.Harami("EURUSD",Resolution.Hour)
        #self.pattern_4hr = self.CandlestickPatterns.Doji("EURUSD",Resolution.Hour)#CandlestickPatterns.Harami("EURUSD",Resolution.Hour)
        
       # self.pattern_hrm = self.CandlestickPatterns.Harami("EURUSD",Resolution.Hour)#CandlestickPatterns.Harami("EURUSD",Resolution.Hour)
       # self.pattern_4hr_hrm = self.CandlestickPatterns.Harami("EURUSD",Resolution.Hour)#CandlestickPatterns.Harami("EURUSD",Resolution.Hour)
        
        
        C1_Con=QuoteBarConsolidator(timedelta(hours=4))
        C1_Con.DataConsolidated += self.On_W1
        self.RegisterIndicator("EURUSD",self.pattern_4hr,C1_Con)
       # self.SubscriptionManager.AddConsolidator("EURUSD", C1_Con)
        self.RegisterIndicator("EURUSD",self.bool_4hr,C1_Con)
        self.RegisterIndicator("EURUSD",self.pattern_4hr_hrm,C1_Con)
        self.SubscriptionManager.AddConsolidator("EURUSD", C1_Con)
        
        self.Consolidate("EURUSD",timedelta(hours=1),lambda x:self.window.Add(x))
        self.Consolidate("EURUSD",timedelta(minutes=5),lambda x:self.windowmin.Add(x))
        
        self.Consolidate("EURUSD",timedelta(hours=4),lambda x:self.window_4hr.Add(x))
        # Register our Handlers
        W1_Con = QuoteBarConsolidator(timedelta(minutes=5))
        W1_Con.DataConsolidated += self.On_W1
        self.RegisterIndicator("EURUSD",self.__macd_min,W1_Con)
        self.SubscriptionManager.AddConsolidator("EURUSD", W1_Con)
        
        self.RegisterIndicator("EURUSD",self.bool_m,W1_Con)
        self.SubscriptionManager.AddConsolidator("EURUSD", W1_Con)
        
       # self.SetRiskManagement(MaximumDrawdownPercentPortfolio(0.02))
       # self.AddRiskManagement(MaximumUnrealizedProfitPercentPerSecurity(0.01))
        
    def On_W1(self,sender,bar):
        '''
        This method will be called every time a new 30 minute bar is ready. 
   
        bar = The incoming Tradebar. This is different to the data object in OnData()
        '''
        #self.Plot('Macd', 'W1', self.__macd.Current.Value)
    def MacdUpdatedMin(self, sender, updated):
        '''Adds updated values to rolling window'''
        self._macdWin2.Add(updated)
        
    def MacdUpdated(self, sender, updated):
        '''Adds updated values to rolling window'''
        self._macdWin.Add(updated)
        
    def OnData(self, data):
       # self.window.Add(data["EURUSD"])
        #self.Consolidate("EURUSD",timedelta(minutes=1),lambda x:self.window.Add(x))
        if not (self.window.IsReady and self._macdWin.IsReady and self._macdWin2.IsReady and self.window_4hr.IsReady and self.adx.IsReady): return
       
        self.fxClose_4hr=self.window_4hr[0].Close
        
        
        Low,High=self.Chand()
        
        Ch_long=High-self.atr.Current.Value*3
        Ch_short=Low+self.atr.Current.Value*3
      
        UP_BB_m=self.bool_m.UpperBand.Current.Value #data["EURUSD"].High
       # bb2 = MA - 2*ST
        DW_BB_m=self.bool_m.LowerBand.Current.Value  #
        
        UP_BB_hr=self.bool_hr.UpperBand.Current.Value #data["EURUSD"].High
       # bb2 = MA - 2*ST
        DW_BB_hr=self.bool_hr.LowerBand.Current.Value  #
        
        
        UP_BB_4hr=self.bool_4hr.UpperBand.Current.Value #data["EURUSD"].High
       # bb2 = MA - 2*ST
        DW_BB_4hr=self.bool_4hr.LowerBand.Current.Value  #
        
       # if self.fxClose >self.lwma_daily.Current.Value:
      #  if (self.fxClose >self.lwma_4hr.Current.Value) or (self.fxClose >self.lwma_hr.Current.Value):
        if (round(self.fxClose_4hr,4)==round(DW_BB_4hr,4)):
                

            signal=self.macd()
            if signal=="BUY":
                if self.adx.Current.Value>25:
                    signad=self.macd_min()
                    if (round(self.fxClose00,4)==round(DW_BB_m,4)) and(signad=="BUY"): 
                        if round(self.fxClose,3)==round(Ch_long,3):
                            self.Liquidate("EURUSD")
                        else:
                            if not self.Portfolio.Invested:
                                self.SetHoldings("EURUSD", 0.02)
       # if (self.fxClose <self.lwma_4hr.Current.Value) or (self.fxClose <self.lwma_hr.Current.Value):
        if (round(self.fxClose_4hr,4)==round(UP_BB_4hr,4)):
               
                signal=self.macd()
                if signal=="SELL":
                    if self.adx.Current.Value>25:
                        signad=self.macd_min()
                        if (round(self.fxClose00,4)==round(UP_BB_m,4)) and(signad=="SELL"):
                            if round(self.fxClose,3)==round(Ch_short,3):
                                self.Liquidate("EURUSD")
                            else:
                                if not self.Portfolio.Invested:
                                    self.SetHoldings("EURUSD",-0.02)
         #self.Liquidate("EURUSD")
       # if not self.Portfolio.Invested and currSma.Value > pastSma.Value:
        #    self.SetHoldings("EURUSD", 1)
        
    def macd(self):
        if not (self.window.IsReady and self._macdWin.IsReady and self._macdWin2.IsReady): return
       # self.fxOpen =data["EURUSD"].Open          
        self.fxOpen=self.window[0].Open
        self.fxClose=self.window[0].Close
        self.pxClose2=self.window[1].Close
        self.pxClose3=self.window[2].Close
        self.pxClose4=self.window[3].Close
        self.pxClose5=self.window[4].Close
        self.pxClose6=self.window[5].Close
        self.pxClose7=self.window[6].Close
        self.pxClose8=self.window[7].Close
        self.pxClose9=self.window[8].Close
        self.pxClose10=self.window[9].Close
        self.pxClose11=self.window[10].Close
        self.pxClose12=self.window[11].Close
        self.pxClose13=self.window[12].Close
        self.pxClose14=self.window[13].Close
        self.pxClose15=self.window[14].Close
        self.pxClose16=self.window[15].Close
        self.pxClose17=self.window[16].Close
        self.pxClose18=self.window[17].Close
        self.pxClose19=self.window[18].Close
        self.pxClose20=self.window[19].Close
        
        #MACD
        self.currmacd = self._macdWin[0]                     # Current SMA had index zero.
        self.pastmacd = self._macdWin[self._macdWin.Count-1]   # Oldest SMA has index of window count minus 1.
        self.pastmacd2 = self._macdWin[self._macdWin.Count-2]
        self.pastmacd3 = self._macdWin[self._macdWin.Count-3]
        self.pastmacd4 = self._macdWin[self._macdWin.Count-4]
        self.pastmacd5 = self._macdWin[self._macdWin.Count-5]
        self.pastmacd6 = self._macdWin[self._macdWin.Count-6]
        self.pastmacd7 = self._macdWin[self._macdWin.Count-7]
        self.pastmacd8 = self._macdWin[self._macdWin.Count-8]
        self.pastmacd9 = self._macdWin[self._macdWin.Count-9]
        self.pastmacd10 = self._macdWin[self._macdWin.Count-10]
        self.pastmacd11 = self._macdWin[self._macdWin.Count-11]
        self.pastmacd12 = self._macdWin[self._macdWin.Count-12]
        self.pastmacd13 = self._macdWin[self._macdWin.Count-13]
        self.pastmacd14 = self._macdWin[self._macdWin.Count-14]
        self.pastmacd15 = self._macdWin[self._macdWin.Count-15]
        self.pastmacd16 = self._macdWin[self._macdWin.Count-16]
        self.pastmacd17 = self._macdWin[self._macdWin.Count-17]
        self.pastmacd18 = self._macdWin[self._macdWin.Count-18]
        self.pastmacd19 = self._macdWin[self._macdWin.Count-19]
        self.pastmacd20 = self._macdWin[self._macdWin.Count-20]
        if not self.__macd.IsReady: return
        data=""
        #data.clear()
        if (self.pastmacd>self.pastmacd2 and self.pastmacd>self.currmacd):
            extremum2=self.pastmacd
            extremum1=max([self.pastmacd2,self.pastmacd3,self.pastmacd4,self.pastmacd5,self.pastmacd6,self.pastmacd7,self.pastmacd8,self.pastmacd9,self.pastmacd10,self.pastmacd11,self.pastmacd12,self.pastmacd13,self.pastmacd14,self.pastmacd15,self.pastmacd16,self.pastmacd17,self.pastmacd18,self.pastmacd19,self.pastmacd20])

            preciomax2=self.fxClose
            preciomax=max([self.pxClose2,self.pxClose3,self.pxClose4,self.pxClose5,self.pxClose6,self.pxClose7,self.pxClose8,self.pxClose9,self.pxClose10,self.pxClose11,self.pxClose12,self.pxClose13,self.pxClose14,self.pxClose15,self.pxClose16,self.pxClose17,self.pxClose18,self.pxClose19,self.pxClose20])

            if (extremum2<extremum1 and preciomax2>preciomax):
               # for i in range(1,self.length):
               #     if self.__macd[i]==extremum1:
                #        zz=i
                signal="SELL"
                data=signal
            elif (extremum2>extremum1 and preciomax2<preciomax):
               # print("MACD1")
                #for i in range(wik,wik+N):
               #     if self.__macd[i]==extremum1:

                #        zz=i

                signal="BUY"
                data=signal

        elif (self.pastmacd<self.pastmacd2 and self.pastmacd<self.currmacd):
            extremum22=self.pastmacd
            extremum11=min([self.pastmacd2,self.pastmacd3,self.pastmacd4,self.pastmacd5,self.pastmacd6,self.pastmacd7,self.pastmacd8,self.pastmacd9,self.pastmacd10,self.pastmacd11,self.pastmacd12,self.pastmacd13,self.pastmacd14,self.pastmacd15,self.pastmacd16,self.pastmacd17,self.pastmacd18,self.pastmacd19,self.pastmacd20])
            preciomin2=self.fxClose
            preciomin=min([self.pxClose2,self.pxClose3,self.pxClose4,self.pxClose5,self.pxClose6,self.pxClose7,self.pxClose8,self.pxClose9,self.pxClose10,self.pxClose11,self.pxClose12,self.pxClose13,self.pxClose14,self.pxClose15,self.pxClose16,self.pxClose17,self.pxClose18,self.pxClose19,self.pxClose20])

            if (extremum22>extremum11 and preciomin2<preciomin):

               # for i2 in range(1,self.length):

                 #   if self.__macd[j][i2]==extremum11:
                       # zz2=i2
                signal="BUY"
                data=signal
            elif (extremum22<extremum11 and preciomin2>preciomin):
                signal="SELL"
                data=signal
                #for i2 in range(wik,wik+N):
                   # if self.__macd[i2]==extremum11:
                        #zz2=i2
                       # signal="SELL"
        return data 
    def macd_min(self):
        self.fxOpen00=self.windowmin[0].Open
        self.fxClose00=self.windowmin[0].Close
        self.pxClose22=self.windowmin[1].Close
        self.pxClose33=self.windowmin[2].Close
        self.pxClose44=self.windowmin[3].Close
        self.pxClose55=self.windowmin[4].Close
        self.pxClose66=self.windowmin[5].Close
        self.pxClose77=self.windowmin[6].Close
        self.pxClose88=self.windowmin[7].Close
        self.pxClose99=self.windowmin[8].Close
        self.pxClose100=self.windowmin[9].Close
        self.pxClose111=self.windowmin[10].Close
        self.pxClose122=self.windowmin[11].Close
        self.pxClose133=self.windowmin[12].Close
        self.pxClose144=self.windowmin[13].Close
        self.pxClose155=self.windowmin[14].Close
        self.pxClose166=self.windowmin[15].Close
        self.pxClose177=self.windowmin[16].Close
        self.pxClose188=self.windowmin[17].Close
        self.pxClose199=self.windowmin[18].Close
        self.pxClose200=self.windowmin[19].Close
        
        #MACD
        self.currmacd00 = self._macdWin2[0]                     # Current SMA had index zero.
        self.pastmacd00 = self._macdWin2[self._macdWin2.Count-1]   # Oldest SMA has index of window count minus 1.
        self.pastmacd22 = self._macdWin2[self._macdWin2.Count-2]
        self.pastmacd33 = self._macdWin2[self._macdWin2.Count-3]
        self.pastmacd44 = self._macdWin2[self._macdWin2.Count-4]
        self.pastmacd55 = self._macdWin2[self._macdWin2.Count-5]
        self.pastmacd66 = self._macdWin2[self._macdWin2.Count-6]
        self.pastmacd77 = self._macdWin2[self._macdWin2.Count-7]
        self.pastmacd88 = self._macdWin2[self._macdWin2.Count-8]
        self.pastmacd99 = self._macdWin2[self._macdWin2.Count-9]
        self.pastmacd100 = self._macdWin2[self._macdWin2.Count-10]
        self.pastmacd111 = self._macdWin2[self._macdWin2.Count-11]
        self.pastmacd122 = self._macdWin2[self._macdWin2.Count-12]
        self.pastmacd133 = self._macdWin2[self._macdWin2.Count-13]
        self.pastmacd144 = self._macdWin2[self._macdWin2.Count-14]
        self.pastmacd155 = self._macdWin2[self._macdWin2.Count-15]
        self.pastmacd166 = self._macdWin2[self._macdWin2.Count-16]
        self.pastmacd177 = self._macdWin2[self._macdWin2.Count-17]
        self.pastmacd188 = self._macdWin2[self._macdWin2.Count-18]
        self.pastmacd199 = self._macdWin2[self._macdWin2.Count-19]
        self.pastmacd200 = self._macdWin2[self._macdWin2.Count-20]
        
        if not self.__macd_min.IsReady: return
        data=""
        #data.clear()
        if (self.pastmacd00>self.pastmacd22 and self.pastmacd00>self.currmacd00):
            extremum2=self.pastmacd00
            extremum1=max([self.pastmacd22,self.pastmacd33,self.pastmacd44,self.pastmacd55,self.pastmacd66,self.pastmacd77,self.pastmacd88,self.pastmacd99,self.pastmacd100,self.pastmacd111,self.pastmacd122,self.pastmacd133,self.pastmacd144,self.pastmacd155,self.pastmacd166,self.pastmacd177,self.pastmacd188,self.pastmacd199,self.pastmacd200])

            preciomax2=self.fxClose00
            preciomax=max([self.pxClose22,self.pxClose33,self.pxClose44,self.pxClose55,self.pxClose66,self.pxClose77,self.pxClose88,self.pxClose99,self.pxClose100,self.pxClose111,self.pxClose122,self.pxClose133,self.pxClose144,self.pxClose155,self.pxClose166,self.pxClose177,self.pxClose188,self.pxClose199,self.pxClose200])

            if (extremum2<extremum1 and preciomax2>preciomax):
               # for i in range(1,self.length):
               #     if self.__macd[i]==extremum1:
                #        zz=i
                signal="SELL"
                data=signal
            elif (extremum2>extremum1 and preciomax2<preciomax):
               # print("MACD1")
                #for i in range(wik,wik+N):
               #     if self.__macd[i]==extremum1:

                #        zz=i

                signal="BUY"
                data=signal

        elif (self.pastmacd00<self.pastmacd22 and self.pastmacd00<self.currmacd00):
            extremum22=self.pastmacd
            extremum11=min([self.pastmacd22,self.pastmacd33,self.pastmacd44,self.pastmacd55,self.pastmacd66,self.pastmacd77,self.pastmacd88,self.pastmacd99,self.pastmacd100,self.pastmacd111,self.pastmacd122,self.pastmacd133,self.pastmacd144,self.pastmacd155,self.pastmacd166,self.pastmacd177,self.pastmacd188,self.pastmacd199,self.pastmacd200])
            preciomin2=self.fxClose
            preciomin=min([self.pxClose2,self.pxClose3,self.pxClose4,self.pxClose5,self.pxClose6,self.pxClose7,self.pxClose8,self.pxClose9,self.pxClose10,self.pxClose11,self.pxClose12,self.pxClose13,self.pxClose14,self.pxClose15,self.pxClose16,self.pxClose17,self.pxClose18,self.pxClose19,self.pxClose20])

            if (extremum22>extremum11 and preciomin2<preciomin):

               # for i2 in range(1,self.length):

                 #   if self.__macd[j][i2]==extremum11:
                       # zz2=i2
                signal="BUY"
                data=signal
            elif (extremum22<extremum11 and preciomin2>preciomin):
                signal="SELL"
                data=signal
                #for i2 in range(wik,wik+N):
                   # if self.__macd[i2]==extremum11:
                        #zz2=i2
                       # signal="SELL"
        return data 
    def Chand(self):
        self.fxHigh1=self.window[0].High
        self.fxHigh2=self.window[1].High
        self.fxHigh3=self.window[2].High
        self.fxHigh4=self.window[3].High
        self.fxHigh5=self.window[4].High
        self.fxHigh6=self.window[5].High
        self.fxHigh7=self.window[6].High
        self.fxHigh8=self.window[7].High
        self.fxHigh9=self.window[8].High
        self.fxHigh10=self.window[9].High
        self.fxHigh11=self.window[10].High
        self.fxHigh12=self.window[11].High
        self.fxHigh13=self.window[12].High
        self.fxHigh14=self.window[13].High
        
        self.HMax=max([self.fxHigh1,self.fxHigh2,self.fxHigh3,self.fxHigh4,self.fxHigh5])
    
        self.fxLow1=self.window[0].Low
        self.fxLow2=self.window[1].Low
        self.fxLow3=self.window[2].Low
        self.fxLow4=self.window[3].Low
        self.fxLow5=self.window[4].Low
        self.fxLow6=self.window[5].Low
        self.fxLow7=self.window[6].Low
        self.fxLow8=self.window[7].Low
        self.fxLow9=self.window[8].Low
        self.fxLow10=self.window[9].Low
        self.fxLow11=self.window[10].Low
        self.fxLow12=self.window[11].Low
        self.fxLow13=self.window[12].Low
        self.fxLow14=self.window[13].Low
        
        self.LMin=min([self.fxLow1,self.fxLow2,self.fxLow3,self.fxLow4,self.fxLow5])
    
        return self.HMax,self.LMin
        
    def OnOrderEvent(self, fill):
        order = self.Transactions.GetOrderById(fill.OrderId)
        self.Log("{0} - {1}:: {2}".format(self.Time, order.Type, fill))